In [13]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_core.tools import tool
from dotenv import load_dotenv
from langchain_core.messages import ToolMessage
import json
load_dotenv()

@tool
def get_restaurant_recommendations(location: str):
    """Provides a list of top restaurant recommendations for a given location."""
    recommendations = {
        "munich": ["Hofbräuhaus", "Augustiner-Keller", "Tantris"],
        "new york": ["Le Bernardin", "Eleven Madison Park", "Joe's Pizza"],
        "paris": ["Le Meurice", "L'Ambroisie", "Bistrot Paul Bert"],
    }
    return recommendations.get(location.lower(), ["No recommendations available for this location."])


tools = [get_restaurant_recommendations]
llm = ChatOpenAI()
llm_with_tools = llm.bind_tools(tools)

messages = [
    HumanMessage("Recommend some restaurants in Munich.")
]

llm_output = llm_with_tools.invoke(messages)

print(f"Step 1, tool_call: {llm_output}")

tool_args = llm_output.tool_calls[0]['args']           # {'location': 'Munich'}
print("tool_args: ",tool_args)
tool_call_id = llm_output.tool_calls[0]['id']
print("tool_call_id: ",tool_call_id)

tool_result = get_restaurant_recommendations(tool_args)
print("tool_result: ",tool_result)
print(f"Step 2, tool_result: {tool_result}")


tool_msg = ToolMessage(
    tool_call_id=tool_call_id,
    content=str(tool_result)
)
print("tool_msg: ", tool_msg)

messages2 = messages + [llm_output] + [tool_msg]
print("messages2: ",messages2)

final_output = llm_with_tools.invoke(messages2)
print("final_output: ",final_output)
print(f"Final answer for user: {final_output.content}")



Step 1, tool_call: content='' additional_kwargs={'tool_calls': [{'id': 'call_rnbgpgMdlKRDGKhrYMYTdFu7', 'function': {'arguments': '{"location":"Munich"}', 'name': 'get_restaurant_recommendations'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 59, 'total_tokens': 78, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None} id='run-5eb138cc-c546-4a43-bdeb-32b0d87ad485-0' tool_calls=[{'name': 'get_restaurant_recommendations', 'args': {'location': 'Munich'}, 'id': 'call_rnbgpgMdlKRDGKhrYMYTdFu7', 'type': 'tool_call'}] usage_metadata={'input_tokens': 59, 'output_tokens': 19, 'total_tokens': 78, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_de